<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-исследования" data-toc-modified-id="Описание-исследования-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание исследования</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Использование-параметра-class_weight" data-toc-modified-id="Использование-параметра-class_weight-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Использование параметра class_weight</a></span></li><li><span><a href="#Техника-downsampling" data-toc-modified-id="Техника-downsampling-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Техника downsampling</a></span><ul class="toc-item"><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Описание исследования

Цель данного исследования состоит в следующем:
 - необходимо подготовить данные для обучения модели
 - обучить модель на несбалансированных классах
 - получить значение метрики f1
 - привести классы к балансу, обучить модель
 - проверить значение метрики f1 с учетом балансировки
 - выбрать лучшую из трех моделей
 - провести тестирование модели

## Подготовка данных

In [1]:
from io import BytesIO
import requests
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_auc_score
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt

In [2]:
RANDOM_STATE = 12345

In [3]:
spreadsheet_id = '1jctWr_hicHv5bJH8BtonBS1JnNB0FDS1q1-cHxyEN2w'
file_name=f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv'
r = requests.get(file_name)
data = pd.read_csv(BytesIO(r.content))

In [4]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Имеется датасет с основной информацией о клиентах банка. Количество объектов 10_000, количество признаков объекта равно 13. Нас интересует признак Exited (целевой признак) - факт ухода клиента. Также можно заметить, что некоторые категориальные признаки необходимые для обучения модели, имеют текстовый вид (Geography, Gender). Их необходимо будет преобразовать.

In [93]:
features_column = [
    'CreditScore', 'Geography', 'Gender', 'Age', 
    'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'   
]
features = data[features_column]
target = data['Exited']
features = pd.get_dummies(features).drop('Gender_Female', axis=1)

Преобразуем категориальные данные из текстового формата в числовой. Применим метод прямого кодирования One-hot-encoding. Теперь данные готовы для обучения модели

In [95]:
features_train, features_others, target_train, target_others = train_test_split(
    features, target, train_size=.6, random_state=RANDOM_STATE
)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_others, target_others, test_size=.5, random_state=RANDOM_STATE
)

print('Доля тренировочных данных: ', features_train.shape[0] / features.shape[0])
print('Доля тестовых данных: ', features_test.shape[0] / features.shape[0])
print('Доля валидационных данных: ', features_valid.shape[0] / features.shape[0])

Доля тренировочных данных:  0.6
Доля тестовых данных:  0.2
Доля валидационных данных:  0.2


Для обучения нам понадобится не все признаки. Например RowNumber, Surname (номер строки и фамилия), можно исключить. По оставшимся признакам будем обучать модель. Также выделим целевой признак Exited (целевой признак). 
Исходную выборку разделим на три: обучающая выборка (train) - она составит 60% от первоначальной, тестовая выборка (test) - она составит 20% от первоначальной, валидационная выборка (valid) - она также составит 20% от первоначальной. 
Для определения успешного предсказания модели мы будем использовать f1 метрику. Эта метрика представляет собой среднее гармоническое между полнотой и точностью правильно предсказанных положительных объектов. Также на каждом этапе изменения модели будем вычислять метрику roc-auc. 

## Исследование задачи

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [97]:
def compute_roc_auc_score(model, features, target):
    propabilities_valid = model.predict_proba(features)
    propabilities_one = propabilities_valid[:, 1]
    print('Показатель метрики roc-auc score:', round(roc_auc_score(target, propabilities_one), 3))

In [98]:
def compute_f1_score(model, features, target):
    predictions = model.predict(features)
    print(f'Показатель метрики f1 score: {round(f1_score(target, predictions), 3)}')

Создадим несколько вспомогательных функций: для подсчета метрики roc-auc, для подсчета метрики f1.

### Модель случайного леса

In [99]:
def get_best_depth_forest(features_train, target_train):
    best_depth_forest = 0
    best_score = 0
    for depth in range(1, 30, 1):
        model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=RANDOM_STATE)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        predictions_valid
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_score = score
            best_depth_forest = depth
    print(f'best_depth: {best_depth_forest}, score: {best_score}')
    return best_depth_forest

In [100]:
best_depth_forest = get_best_depth_forest(features_train, target_train)

best_depth: 14, score: 0.5532544378698224


In [101]:
model = RandomForestClassifier(n_estimators=20, max_depth=best_depth_forest, random_state=RANDOM_STATE)
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.847


Для модели случайного леса найдем наилучший показатель для параметра максимальной глубины.  Для этого обучем модель при разных показателях параметра. Наилучшим значением является 14. Метрика roc-auc при таком значении параметра max_depth будет равняться 0.847

In [105]:
def get_best_estimators(features_train, target_train, best_depth_forest):
    best_estimators = 0
    best_score = 0
    for estimators in range(5, 110, 5):
        model = RandomForestClassifier(n_estimators=estimators, max_depth=best_depth_forest, random_state=RANDOM_STATE)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_score = score
            best_estimators = estimators
    print(f'best_estimators: {best_estimators}, best_score: {best_score}')
    return best_estimators

In [103]:
best_estimators = get_best_estimators(features_train, target_train, best_depth_forest)
best_estimators = 15

best_estimators: 20, best_score: 0.5532544378698224


Теперь определим наилучшее значение деревьев. Для этого также как и выше обучи модель при различных значениях этого параметра. Наилучший показатель метрики f1 будет при значении равном 20.

In [104]:
model = RandomForestClassifier(n_estimators=best_estimators, max_depth=best_depth_forest, random_state=RANDOM_STATE)
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.847
Показатель метрики f1 score: 0.553


Определим показатели метрик roc-auc и f1 для модели с наилучшими гиперпараметрами.

### Дерево решений

In [106]:
def get_best_depth_tree(features_train, target_train):
    best_depth_tree = 0
    best_score = 0
    for depth in range(1, 30, 1):
        model = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_STATE)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        predictions_valid
        score = f1_score(target_valid, predictions_valid)
        if score > best_score:
            best_score = score
            best_depth_tree = depth
    print(f'best_depth: {best_depth_tree}, score: {best_score}')
    return best_depth_tree

In [107]:
best_depth_tree = get_best_depth_tree(features_train, target_train)
best_depth_tree = 7

best_depth: 6, score: 0.5253456221198156


Рассмотрим модель дерева решений. Для этой модели подберем наилучший параметр максимальной глубины. Наилучшим значением гиперпараметра max_depth будет являться 6

In [108]:
model = DecisionTreeClassifier(max_depth=best_depth_tree, random_state=RANDOM_STATE)
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.836
Показатель метрики f1 score: 0.525


Определим показатели метрик roc-auc и f1 для модели с наилучшими гиперпараметрами. Данные показатели хуже чем у предыдущей модели - случайный лес

### Логистическая регрессия

In [109]:
model = LogisticRegression(random_state=RANDOM_STATE)
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.642
Показатель метрики f1 score: 0.137


Значение показателей метрик roc-auc и f1 для логистической регресси самый низкие из всех трех моделей: roc-auc - 0.642, f1 - 0.137

Были рассчитаны метрики roc-auc, f1 на трех разных моделях с наилучшими гиперпараметрами для этих моделей. Исходя из значений метрик, наилучшей моеделью является "Модель случайного леса". Однако показатель метрики f1 все равно далек от ожидаемого результата (0.59). Также можно заметить, что чем выше показатель метрики f1 тем, выше показатель метрики roc-auc

## Борьба с дисбалансом

In [110]:
def get_balance(target):
    target_part = target[target == 1].shape[0] / target.shape[0]
    print(f'Доля объектов положительного класса {round(target_part, 3)}')
    print(f'Доля объектов отрицательного класса {round(1 - target_part, 3)}')

In [111]:
get_balance(target_train)

Доля объектов положительного класса 0.199
Доля объектов отрицательного класса 0.801


Проверим классы целевого признака обучающей выборки на дисбаланс. Доля наиболее часто встречающегося класса равна 0.801. Необходимо уменьшить дисбаланс, чтобы данный показатель был близок к 50%. Возможно сделав количество классов равным друг другу, удастся увеличить показатель метрики f1

### Использование параметра class_weight

In [112]:
model = RandomForestClassifier(
    n_estimators=best_estimators, max_depth=best_depth_forest, 
    random_state=RANDOM_STATE, class_weight='balanced'
)
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.844
Показатель метрики f1 score: 0.553


In [113]:
model = DecisionTreeClassifier(
    max_depth=best_depth_tree, 
    random_state=RANDOM_STATE, class_weight='balanced'
)
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.822
Показатель метрики f1 score: 0.566


In [114]:
model = LogisticRegression(class_weight='balanced')
model.fit(features_train, target_train)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.679
Показатель метрики f1 score: 0.429


Попробуем сбалансировать классы. У всех рассмотренных нами ранее моделей есть гиперпараметр, отвечающий за баланс классов. При значении данного параметра равного 'balanced' редкому классу будет задан ранг выше, чем у часто встречающегося класса. Оценив результаты можно сказать, что значительные изменения произошли для моделей регрессии, её показатель f1 стал равен 0.429, и модели дерева решений, её показатель f1 вырос до 0.566 однако этого все равно не достаточно.

### Техника downsampling

In [115]:
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]
features_downsampled = pd.concat([features_zeros.sample(frac=.3, random_state=RANDOM_STATE)] + [features_ones])
target_downsampled = pd.concat([target_zeros.sample(frac=.3, random_state=RANDOM_STATE)] + [target_ones])

features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=RANDOM_STATE)

In [116]:
get_balance(target_downsampled)

Доля объектов положительного класса 0.454
Доля объектов отрицательного класса 0.546


Попробуем убрать дисбаланс классов с помощью техники уменьшения выборки (downsampling). Разделим все признаки тренировочной выборки на два класса отрицательные и положительные, затем сделаем количество признаков с отрицательным классом примерно равным количеству положительных классов, объединим их в один набор данных и перемешаем - таким образом мы получим новый, уменьшенный набор признаков, но уже без сильного дисбаланса классов. Тоже самое сделаем с целевыми признаками. Проверим долю каждого класса по отношению к новой (уменьшенной) обучающей выборке. Теперь доли классов приблизительно равны:
Доля объектов положительного класса 0.454
Доля объектов отрицательного класса 0.546

#### Модель случайного леса

In [117]:
best_depth_forest = get_best_depth_forest(features_downsampled, target_downsampled)

best_depth: 9, score: 0.597938144329897


In [118]:
model = RandomForestClassifier(n_estimators=20, max_depth=best_depth_forest, random_state=RANDOM_STATE)
model.fit(features_downsampled, target_downsampled)
compute_roc_auc_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.84


In [119]:
best_estimators = get_best_estimators(features_downsampled, target_downsampled, best_depth_forest)

best_estimators: 20, best_score: 0.597938144329897


In [120]:
model = RandomForestClassifier(n_estimators=best_estimators, max_depth=best_depth_forest, random_state=RANDOM_STATE)
model.fit(features_downsampled, target_downsampled)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.84
Показатель метрики f1 score: 0.598


#### Дерево решений

In [121]:
best_depth_tree = get_best_depth_tree(features_downsampled, target_downsampled)

best_depth: 5, score: 0.5834230355220668


In [122]:
model = DecisionTreeClassifier(max_depth=best_depth_tree, random_state=RANDOM_STATE)
model.fit(features_downsampled, target_downsampled)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.829
Показатель метрики f1 score: 0.583


#### Логистическая регрессия

In [123]:
model = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced')
model.fit(features_downsampled, target_downsampled)
compute_roc_auc_score(model, features_valid, target_valid)
compute_f1_score(model, features_valid, target_valid)

Показатель метрики roc-auc score: 0.678
Показатель метрики f1 score: 0.427


Обучим модели случайного леса, дерева решений, и логистической регрессии, на новых данных, учитывающих дисбаланс и проверим увеличились ли показатели метрик f1 и roc-auc. В ходе исследования, мы видим, что у всех трех моделей увеличились показатели метрик f1 и roc-auc. Однако только у модели  случайного леса показатель f1 соответствует нашим требованием и превышает значение 0.59: показатель метрики f1 score = 0.598,  показатель метрики roc-auc score = 0.84

## Тестирование модели

In [124]:
model = RandomForestClassifier(n_estimators=best_estimators, max_depth=best_depth_forest, random_state=RANDOM_STATE)
model.fit(features_downsampled, target_downsampled)
compute_roc_auc_score(model, features_test, target_test)
compute_f1_score(model, features_test, target_test)

Показатель метрики roc-auc score: 0.836
Показатель метрики f1 score: 0.608


Проведем тестирование модели с наилучшим показателем метрики f1 - модель случайного леса. Проверку будем осуществлять на третьей выборке - тестовом наборе данных (test). Создав модель с наилучшими значениями гиперпараметров и посчитав метрики на этой моделе мы видим, что показатели даже немного превосходят те, что мы получили на валидационных данных. Следовательно наша модель прошла проверку.

Посчитаем метрику auc-roc (площадь под roc-кривой). Показатель этой метрики равен 0.751. Этот метрика показывает нам долю верно предсказанных пар объектов: (объект класса 1, объекта класса 2)

## Вывод

В ходе исследования было сделано следующее:

- из выборки были выделены признаки:
    CreditScore — кредитный рейтинг
    Geography — страна проживания
    Gender — пол
    Age — возраст
    Tenure — сколько лет человек является клиентом банка
    Balance — баланс на счёте
    NumOfProducts — количество продуктов банка, используемых клиентом
    HasCrCard — наличие кредитной карты
    IsActiveMember — активность клиента
    EstimatedSalary — предполагаемая зарплата
    и целевой признак Exited — факт ухода клиента
- вся выборка была разделена на три части: обучающая выборка (train) - она составит 60% от первоначальной, тестовая выборка (test) - она составит 20% от первоначальной, валидационная выборка (valid) - она также составит 20% от первоначальной.
- категориальные признаки были закодированы способом One-hot-encoding
- был осуществлен поиск показателей метрик f1, roc-auc без учета дисбаланса классов, на трех моделях:
    - Модель случайного леса
    - Дерево решений
    - Логистическая регрессия
- также для этих моделей были найдены лучшие показатели гиперпараметров, при которых мы получили наивысшие значения метрик высшие значения метрик.
- были вычислены следующие показатели:
    - модель случайный лес: показатель метрики roc-auc score 0.848, показатель метрики f1 score: 0.557
    - модель дерево решений: показатель метрики roc-auc score 0.836, показатель метрики f1 score: 0.531
    - модель логистическая регрессия: показатель метрики roc-auc score 0.642, показатель метрики f1 score: 0.137
- чтобы улучшить показатель метрики f1, была сделана перебалансровка классов
- изначальная доля классов по сравнению с тренировочным набором данных целевого признака составила: доля объектов положительного класса 0.199, доля объектов отрицательного класса 0.801
- для борьбы с дисбалансом было использовано два метода
- первый метод заключался в том, что для моделей мы задали параметр class_weight со значением 'balanced'. Этот параметр задает редкому классу ранг выше, чем у часто встречающегося класса. Особого результата метод не дал. Показатели метрик следующие:
     - модель случайный лес: показатель метрики roc-auc score 0.845, показатель метрики f1 score: 0.503
     - модель дерево решений: показатель метрики roc-auc score 0.822, показатель метрики f1 score: 0.566
     - модель логистическая регрессия: показатель метрики roc-auc score 0.679, показатель метрики f1 score: 0.429
- второй метод заключался в уменьшении тестовой выборки (техника downsampling), и приведению доли часто встречающегося класса к доле редкого класса. После использования этого  метода для модели были обучены с использованием новых (сбалансированных классов), а также для них были подобраны, новые значения гиперпараметров.
    - модель случайный лес: показатель метрики roc-auc score 0.849, показатель метрики f1 score: 0.603
    - модель дерево решений: показатель метрики roc-auc score 0.829, показатель метрики f1 score: 0.583
    - модель логистическая регрессия: показатель метрики roc-auc score 0.678, показатель метрики f1 score: 0.427
- модели  случайного леса показала наилучший результат метрики f1 - 0.603
- также было проведено тестирование модели с наилучшим показателем метрики f1.. Проверка осуществлялась на третьем - тестовом наборе данных (test). Создав модель с наилучшими значениями гиперпараметров и посчитав метрики на этой моделе мы убедились, удовлетворяют нашим требованием. Значение метрики f1 составило 0.608


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*